## Web ブラウジングと要約のための Bring Your Own Browser (BYOB) ツールの構築

**免責事項: このクックブックは教育目的のみです。Web 検索とスクレイピング技術を使用する際は、適用されるすべての法律とサービス利用規約に従うことを確認してください。このクックブックでは、概念を説明するために openai.com ドメインに検索を制限して公開情報を取得します。**

GPT-4o などの大規模言語モデル（LLM）には知識のカットオフ日があり、その時点以降に発生した出来事に関する情報が不足しています。最新のデータが不可欠なシナリオでは、正確で関連性の高い応答を確保するために、LLM に現在の Web 情報へのアクセスを提供する必要があります。

このガイドでは、この制限を克服するために Python を使用して Bring Your Own Browser (BYOB) ツールを構築します。私たちの目標は、OpenAI による最新の製品発表などの最新の開発を含む、アプリケーションで最新の回答を提供するシステムを作成することです。Web 検索機能を LLM と統合することで、オンラインで利用可能な最新情報に基づいてモデルが応答を生成できるようにします。

公開されている任意の検索 API を使用できますが、Web 検索を実行するために Google の Custom Search API を利用します。検索結果から取得した情報は処理され、Retrieval-Augmented Generation (RAG) を通じて最終的な応答を生成するために LLM に渡されます。

**Bring Your Own Browser (BYOB)** ツールを使用すると、ユーザーはプログラム的に Web ブラウジングタスクを実行できます。このノートブックでは、以下の機能を持つ BYOB ツールを作成します：

**#1. 検索エンジンの設定:** Google の Custom Search API などの公開検索 API を使用して Web 検索を実行し、関連する検索結果のリストを取得します。

**#2. 検索辞書の構築:** 検索結果から各 Web ページのタイトル、URL、および要約を収集して、構造化された情報辞書を作成します。

**#3. RAG 応答の生成:** 収集した情報を LLM に渡すことで Retrieval-Augmented Generation (RAG) を実装し、ユーザーのクエリに対する最終的な応答を生成します。

### 使用例
このクックブックでは、OpenAIの最近の製品リリースを時系列順にリストアップしたいユーザーの例を取り上げます。現在のGPT-4oモデルには知識のカットオフ日があるため、2024年9月にリリースされたo1-previewモデルなどの最近の製品リリースについてモデルが知っているとは期待できません。

In [1]:
from openai import OpenAI

client = OpenAI()

search_query = "List the latest OpenAI product launches in chronological order from latest to oldest in the past 2 years"


response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful agent."},
        {"role": "user", "content": search_query}]
).choices[0].message.content

print(response)

OpenAI has had several notable product launches and updates in the past couple of years. Here’s a chronological list of some significant ones:

1. **ChatGPT (November 2022)**: OpenAI launched an AI chatbot called ChatGPT, which is based on GPT-3.5. This chatbot became widely popular due to its impressive capabilities in understanding and generating human-like text.

2. **GPT-4 (March 2023)**: OpenAI released GPT-4, the latest version of their Generative Pre-trained Transformer model. It brought improvements in both performance and accuracy over its predecessors.

3. **DALL-E 2 (April 2022)**: The second version of DALL-E, an AI model that can generate images from textual descriptions, was launched with enhanced image resolution and more robust capabilities.

4. **Whisper (September 2022)**: Whisper, an automatic speech recognition (ASR) system, was introduced. This model can handle multiple languages and is useful for transcribing and understanding spoken language.

These are some of t

知識のカットオフを考慮すると、予想通りモデルはOpenAIによる最近の製品リリースについて知らないことがわかります。

### BYOBツールの設定
モデルに最新のイベント情報を提供するために、以下の手順に従います：

##### ステップ1: ウェブ検索結果を提供する検索エンジンの設定
##### ステップ2: ウェブページのタイトル、URL、要約を含む検索辞書の構築
##### ステップ3: ユーザークエリに対するRAGレスポンスを生成するためにモデルに情報を渡す


開始する前に、以下が準備されていることを確認してください：マシンに**Python 3.12以降**がインストールされていること。また、Google Custom Search APIキーとCustom Search Engine ID（CSE ID）が必要です。必要なPythonパッケージがインストールされていること：`requests`、`beautifulsoup4`、`openai`。そして、OPENAI_API_KEYが環境変数として設定されていることを確認してください。

#### ステップ1: ウェブ検索結果を提供する検索エンジンの設定
このタスクを実行するために、公開されているウェブ検索APIを使用できます。GoogleのCustom Search APIを使用してカスタム検索エンジンを設定します。このエンジンは、ユーザーのクエリに基づいて関連するウェブページのリストを取得し、最新かつ適切な結果の取得に焦点を当てます。

**a. Search APIキーと関数の設定:** Google Developers ConsoleからGoogle APIキーとCustom Search Engine ID（CSE ID）を取得します。APIキーとCustom Search Engine ID（CSE ID）の設定については、この[Programmable Search Engineリンク](https://developers.google.com/custom-search/v1/overview)にアクセスして設定できます。

以下の`search`関数は、検索用語、APIとCSE IDキー、および返す検索結果の数に基づいて検索を設定します。出力を`openai.com`のみに制限するために`site_filter`パラメータを導入します。

In [2]:
import requests  # For making HTTP requests to APIs and websites

def search(search_item, api_key, cse_id, search_depth=10, site_filter=None):
    service_url = 'https://www.googleapis.com/customsearch/v1'

    params = {
        'q': search_item,
        'key': api_key,
        'cx': cse_id,
        'num': search_depth
    }

    try:
        response = requests.get(service_url, params=params)
        response.raise_for_status()
        results = response.json()

        # Check if 'items' exists in the results
        if 'items' in results:
            if site_filter is not None:
                
                # Filter results to include only those with site_filter in the link
                filtered_results = [result for result in results['items'] if site_filter in result['link']]

                if filtered_results:
                    return filtered_results
                else:
                    print(f"No results with {site_filter} found.")
                    return []
            else:
                if 'items' in results:
                    return results['items']
                else:
                    print("No search results found.")
                    return []

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the search: {e}")
        return []


**b. 検索エンジン用の検索語句を特定する：** サードパーティAPIから特定の結果を取得する前に、ブラウザ検索APIが取得すべき特定の用語を特定するためにクエリ拡張を使用する必要がある場合があります。**クエリ拡張**とは、関連する用語、同義語、またはバリエーションを追加することで、元のユーザークエリを拡張するプロセスです。この技術は、GoogleのCustom Search APIなどの検索エンジンが、ユーザーが使用する自然言語プロンプトだけでなく、関連する用語の範囲をマッチングすることに優れているため、不可欠です。

例えば、生のクエリ`"List the latest OpenAI product launches in chronological order from latest to oldest in the past 2 years"`のみで検索すると、`"Latest OpenAI product launches"`のような簡潔なフレーズでのより具体的で直接的な検索よりも、結果が少なく関連性が低くなる可能性があります。以下のコードでは、ユーザーの元の`search_query`を使用して、Google APIで結果を取得するために使用するより具体的な検索語句を生成します。

In [3]:
search_term = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Provide a google search term based on search query provided below in 3-4 words"},
        {"role": "user", "content": search_query}]
).choices[0].message.content

print(search_term)

Latest OpenAI product launches


**c. 検索機能を呼び出す：** 検索語句を取得したので、Google検索APIから結果を取得するために検索機能を呼び出します。この時点では、結果にはWebページのリンクとスニペットのみが含まれています。次のステップでは、Webページからより多くの情報を取得し、それを辞書形式でまとめてモデルに渡します。

In [4]:
from dotenv import load_dotenv
import os

load_dotenv('.env')

api_key = os.getenv('API_KEY')
cse_id = os.getenv('CSE_ID')

search_items = search(search_item=search_term, api_key=api_key, cse_id=cse_id, search_depth=10, site_filter="https://openai.com")


In [5]:
for item in search_items:
    print(f"Link: {item['link']}")
    print(f"Snippet: {item['snippet']}\n")

Link: https://openai.com/news/
Snippet: Overview ; Product. Sep 12, 2024. Introducing OpenAI o1 ; Product. Jul 25, 2024. SearchGPT is a prototype of new AI search features ; Research. Jul 18, 2024. GPT- ...

Link: https://openai.com/index/new-models-and-developer-products-announced-at-devday/
Snippet: Nov 6, 2023 ... GPT-4 Turbo with 128K context · We released the first version of GPT-4 in March and made GPT-4 generally available to all developers in July.

Link: https://openai.com/news/product/
Snippet: Discover the latest product advancements from OpenAI and the ways they're being used by individuals and businesses.

Link: https://openai.com/
Snippet: A new series of AI models designed to spend more time thinking before they respond. Learn more · (opens in a new window) ...

Link: https://openai.com/index/sora/
Snippet: Feb 15, 2024 ... We plan to include C2PA metadata(opens in a new window) in the future if we deploy the model in an OpenAI product. In addition to us developing ...



#### ステップ2: ウェブページのタイトル、URL、要約を含む検索辞書の構築
検索結果を取得した後、関連情報を抽出・整理して、最終的な出力のためにLLMに渡せるようにします。

**a. ウェブページコンテンツのスクレイピング:** 検索結果の各URLについて、ウェブページを取得してテキストコンテンツを抽出し、`retrieve_content`関数で示されているように、スクリプトや広告などの無関係なデータを除外します。

**b. コンテンツの要約:** LLMを使用してスクレイピングしたコンテンツの簡潔な要約を生成し、ユーザーのクエリに関連する情報に焦点を当てます。`summarize_content`関数で概説されているように、モデルには元の検索テキストを提供することで、検索意図に合わせたコンテンツの要約に集中できます。

**c. 構造化辞書の作成:** データを辞書またはDataFrameに整理し、各ウェブページのタイトル、リンク、要約を含むようにします。この構造は、適切な引用を含む要約を生成するためにLLMに渡すことができます。

In [6]:
import requests
from bs4 import BeautifulSoup

TRUNCATE_SCRAPED_TEXT = 50000  # Adjust based on your model's context window
SEARCH_DEPTH = 5

def retrieve_content(url, max_tokens=TRUNCATE_SCRAPED_TEXT):
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            for script_or_style in soup(['script', 'style']):
                script_or_style.decompose()

            text = soup.get_text(separator=' ', strip=True)
            characters = max_tokens * 4  # Approximate conversion
            text = text[:characters]
            return text
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve {url}: {e}")
            return None
        
def summarize_content(content, search_term, character_limit=500):
        prompt = (
            f"You are an AI assistant tasked with summarizing content relevant to '{search_term}'. "
            f"Please provide a concise summary in {character_limit} characters or less."
        )
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": content}]
            )
            summary = response.choices[0].message.content
            return summary
        except Exception as e:
            print(f"An error occurred during summarization: {e}")
            return None

def get_search_results(search_items, character_limit=500):
    # Generate a summary of search results for the given search term
    results_list = []
    for idx, item in enumerate(search_items, start=1):
        url = item.get('link')
        
        snippet = item.get('snippet', '')
        web_content = retrieve_content(url, TRUNCATE_SCRAPED_TEXT)
        
        if web_content is None:
            print(f"Error: skipped URL: {url}")
        else:
            summary = summarize_content(web_content, search_term, character_limit)
            result_dict = {
                'order': idx,
                'link': url,
                'title': snippet,
                'Summary': summary
            }
            results_list.append(result_dict)
    return results_list

In [7]:
results = get_search_results(search_items)

for result in results:
    print(f"Search order: {result['order']}")
    print(f"Link: {result['link']}")
    print(f"Snippet: {result['title']}")
    print(f"Summary: {result['Summary']}")
    print('-' * 80)

Search order: 1
Link: https://openai.com/news/
Snippet: Overview ; Product. Sep 12, 2024. Introducing OpenAI o1 ; Product. Jul 25, 2024. SearchGPT is a prototype of new AI search features ; Research. Jul 18, 2024. GPT- ...
Summary: OpenAI recently launched several notable products in 2024, including OpenAI o1 and SearchGPT, a prototype for enhanced AI search capabilities. Additionally, GPT-4o mini was introduced, enhancing cost-efficient intelligence. The organization also rolled out OpenAI for Nonprofits and ChatGPT Edu to support various sectors. Improvements in data analysis within ChatGPT and enhancements to the fine-tuning API were also announced. These updates reflect OpenAI's ongoing commitment to advancing AI technologies across different fields.
--------------------------------------------------------------------------------
Search order: 2
Link: https://openai.com/index/new-models-and-developer-products-announced-at-devday/
Snippet: Nov 6, 2023 ... GPT-4 Turbo with 128K conte

最新の結果を取得しました。（注意：これらの結果は、このスクリプトを実行するタイミングによって異なります。）

#### ステップ3: 情報をモデルに渡してユーザークエリに対するRAGレスポンスを生成する
検索データがJSONデータ構造で整理されたら、この情報を元のユーザークエリと共にLLMに渡して最終的なレスポンスを生成します。これにより、LLMのレスポンスには元の知識カットオフを超えた情報が含まれ、最新の洞察を提供できます。

In [8]:
import json 

final_prompt = (
    f"The user will provide a dictionary of search results in JSON format for search query {search_term} Based on on the search results provided by the user, provide a detailed response to this query: **'{search_query}'**. Make sure to cite all the sources at the end of your answer."
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": final_prompt},
        {"role": "user", "content": json.dumps(results)}],
    temperature=0

)
summary = response.choices[0].message.content

print(summary)

Based on the search results provided, here is a chronological list of the latest OpenAI product launches from the past two years, ordered from the most recent to the oldest:

1. **September 12, 2024**: **OpenAI o1**
   - A versatile AI tool designed to enhance reasoning capabilities.
   - Source: [OpenAI News](https://openai.com/news/)

2. **July 25, 2024**: **SearchGPT**
   - A prototype aimed at enhancing AI-driven search capabilities.
   - Source: [OpenAI News](https://openai.com/news/)

3. **July 18, 2024**: **GPT-4o mini**
   - A cost-efficient intelligence model.
   - Source: [OpenAI News](https://openai.com/news/)

4. **May 2024**: **OpenAI for Education**
   - Focuses on integrating AI into educational settings.
   - Source: [OpenAI News](https://openai.com/news/product/)

5. **February 15, 2024**: **Sora**
   - An AI model capable of generating high-quality text-to-video content.
   - Source: [OpenAI Sora](https://openai.com/index/sora/)

6. **November 6, 2023**: **GPT-4 Turbo

### 結論

大規模言語モデル（LLM）には知識のカットオフがあり、最近の出来事を認識していない可能性があります。最新の情報を提供するために、Pythonを使用してBring Your Own Browser（BYOB）ツールを構築できます。このツールは現在のWebデータを取得し、LLMに供給することで、最新の応答を可能にします。

このプロセスには3つの主要なステップが含まれます：

**#1 検索エンジンの設定：** GoogleのCustom Search APIなどの公開検索APIを使用してWeb検索を実行し、関連する検索結果のリストを取得します。

**#2 検索辞書の構築：** 検索結果から各Webページのタイトル、URL、要約を収集し、構造化された情報辞書を作成します。

**#3 RAG応答の生成：** 収集した情報をLLMに渡すことでRetrieval-Augmented Generation（RAG）を実装し、LLMがユーザーのクエリに対する最終的な応答を生成します。

これらのステップに従うことで、OpenAIの最新製品リリースなどの最新の動向を含む、最新の回答を提供するLLMの能力をアプリケーションで向上させることができます。